In [41]:
import warnings 
warnings.filterwarnings('ignore')
from sk_embed_generation_draft02 import EmbeddingGenerator
from sk_model import SkipGramModel
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import itertools 
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, log_loss, roc_curve, auc, roc_auc_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
stem_data = EmbeddingGenerator('/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/STU_CRS_TBL_full.csv',
                              '/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/student_attribute_table_full.xlsx',
                              '/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/STUDENT_RETENTION_TABLE_full.csv',
                              '/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/stem_majors.csv')

In [3]:
model = SkipGramModel(stem_data.embedding_id, stem_data.train_students, stem_data.valid_students)

Model: "Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Course_ids (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Context (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
Course_embedding (Embedding)    (None, 1, 78)        566670      Course_ids[0][0]                 
__________________________________________________________________________________________________
Context_embedding (Embedding)   (None, 1, 78)        566670      Context[0][0]                    
______________________________________________________________________________________________

In [4]:
## top model.load is the sample data
model.load('/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/skip_gram_weights/skip_gram_78dim_lr01_sampleData.hdf5')

## below model.load uses the full data
#model.load('/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/skip_gram_weights/skip_gram_218dim_lr02879_fullData.hdf5')

In [ ]:
#model.train(stem_data)

In [ ]:
## the inference here is there is a 27% chance course 5 is taken with course 10, and a 24% chance course 6 is 
## taken with course 6
#### is it worth plotting  ?  how can we generalize this a little to visualize it.
#model.model.predict([np.array([[5],[6]]),np.array([[10],[6]])])

In [5]:
## pulling out the embeddings to prep for comparing vectors using only the embeddings get_embeddings() gives 
## back embedding matrix
#model_embeddings = model.get_embedding()
stem_data.set_embedding(model.get_embedding())

In [ ]:
# demographic_models_KNN = [KNeighborsClassifier(n_neighbors=11), KNeighborsClassifier(n_neighbors=11), 
#                       KNeighborsClassifier(n_neighbors=11), KNeighborsClassifier(n_neighbors=11)]

In [ ]:
# demographic_models_NB = [GaussianNB(var_smoothing=1e1), GaussianNB(var_smoothing=1e1), 
#                         GaussianNB(var_smoothing=1e1), GaussianNB(var_smoothing=1e1)]

In [ ]:
# demo_xtrain, demo_ytrain = stem_data.get_demographic('train')
# demo_xvalid, demo_yvalid = stem_data.get_demographic('valid')

In [ ]:
# demo_xvalid = demo_xvalid.reshape(-1,218)
# demo_xtrain = demo_xtrain.reshape(-1,218)
# demo_ytrain = demo_ytrain.reshape(-1,4)
# demo_yvalid = demo_yvalid.reshape(-1,4)

In [ ]:
# for i in range(4):
#     demographic_models_NB[i].fit(demo_xtrain, demo_ytrain[:,i])  
#     print(demographic_models_NB[i].score(demo_xvalid, demo_yvalid[:,i]))

In [ ]:
# len(np.unique((demo_ytrain[:,0])))

In [ ]:
# ethnicity_model = demographic_models_NB[0]
# y_pred = ethnicity_model.predict(demo_xvalid)
# y_proba = ethnicity_model.predict_proba(demo_xvalid)

# ethnicity_model_confusion_matrix = pd.DataFrame(confusion_matrix(demo_yvalid[:,0], y_pred))
# ethnicity_model_confusion_matrix

In [ ]:
# ethnicity_prior_only_model = np.repeat(ethnicity_model.class_prior_.reshape(1,-1), 
#                                        len(demo_yvalid), axis=0)

# print(ethnicity_prior_only_model.shape)
# print(y_proba.shape)

# print(log_loss(demo_yvalid[:,0], y_proba))
# print(log_loss(demo_yvalid[:,0], ethnicity_prior_only_model))

In [ ]:
# #plt.figure(figsize=(12,10))
# plt.title('Ethnicity model y_pred density')
# ethn_density = sb.distplot(y_pred)

In [ ]:
# plt.title('Ethnicity model y_true density')
# ethn_density = sb.distplot(demo_ytrain[:,0])

In [ ]:
# urm_flag_model = demographic_models_NB[1]
# y_pred = urm_flag_model.predict(demo_xvalid)
# y_proba = urm_flag_model.predict_proba(demo_xvalid)

# urm_flag_model_confusion_matrix = pd.DataFrame(confusion_matrix(demo_yvalid[:,1], y_pred))
# urm_flag_model_confusion_matrix

In [ ]:
# urm_flag_prior_only_model = np.repeat(urm_flag_model.class_prior_.reshape(1,-1), 
#                                        len(demo_yvalid), axis=0)

# print(urm_flag_prior_only_model.shape)
# print(y_proba.shape)

# print(log_loss(demo_yvalid[:,1], y_proba))
# print(log_loss(demo_yvalid[:,1], urm_flag_prior_only_model))

In [ ]:
# plt.title('urm_flag model y_pred density')
# urm_flag_density = sb.distplot(y_pred)

In [ ]:
# plt.title('urm_flag model y_true density')
# urm_flag_density = sb.distplot(demo_ytrain[:,1])

In [ ]:
# gender_model = demographic_models_NB[2]
# y_pred = gender_model.predict(demo_xvalid)
# y_proba = gender_model.predict_proba(demo_xvalid)

# gender_model_confusion_matrix = pd.DataFrame(confusion_matrix(demo_yvalid[:,2], y_pred))
# gender_model_confusion_matrix

In [ ]:
# gender_prior_only_model = np.repeat(gender_model.class_prior_.reshape(1,-1), 
#                                        len(demo_yvalid), axis=0)

# print(gender_prior_only_model.shape)
# print(y_proba.shape)

# print(log_loss(demo_yvalid[:,2], y_proba))
# print(log_loss(demo_yvalid[:,2], gender_prior_only_model))

In [ ]:
# plt.title('urm_flag model y_pred density')
# gender_density = sb.distplot(y_pred)

In [ ]:
# plt.title('urm_flag model y_true density')
# urm_flag_density = sb.distplot(demo_ytrain[:,2])

In [ ]:
# pell_elig_model = demographic_models_NB[3]
# y_pred = pell_elig_model.predict(demo_xvalid)
# y_proba = pell_elig_model.predict_proba(demo_xvalid)

# pell_elig_model_confusion_matrix = pd.DataFrame(confusion_matrix(demo_yvalid[:,3], y_pred))
# pell_elig_model_confusion_matrix

In [ ]:
# pell_elig_prior_only_model = np.repeat(pell_elig_model.class_prior_.reshape(1,-1), 
#                                        len(demo_yvalid), axis=0)

# print(pell_elig_prior_only_model.shape)
# print(y_proba.shape)

# print(log_loss(demo_yvalid[:,3], y_proba))
# print(log_loss(demo_yvalid[:,3], pell_elig_prior_only_model))

In [ ]:
# plt.title('pell eligibility model y_pred density')
# pell_elig_density = sb.distplot(y_pred)

In [ ]:
# plt.title('pell eligibility model y_true density')
# pell_elig_density = sb.distplot(demo_ytrain[:,3])

In [ ]:
# ethnicity_mapping = {'Refused to Answer':'NOANSWER', 'Not Applicable':'NOANSWER', 'OTHRAMER':'Other American',
#                     'American Indian/Alaska Native':'Native', 'Native Hawaiian/Pacific Island':'Native',
#                     'Native Hawaiian/Oth Pac Island':'Native'}

# stem_data.student_attr_df['ETHNICITY'] = stem_data.student_attr_df['ETHNICITY'].replace(ethnicity_mapping)

In [ ]:
# print(stem_data.student_attr_df['ETHNICITY'].isnull().value_counts())
# stem_data.student_attr_df['ETHNICITY'].nunique()
# stem_data.student_attr_df['ETHNICITY'].count()
# stem_data.student_attr_df['ETHNICITY'].value_counts()

In [ ]:
# print(stem_data.student_attr_df['URM_FLAG'].value_counts())
# for i in stem_data.student_attr_df['URM_FLAG'].value_counts():
#     print(i/stem_data.student_attr_df['URM_FLAG'].count())

In [ ]:
# print(stem_data.student_attr_df['GENDER'].value_counts())
# for i in stem_data.student_attr_df['GENDER'].value_counts():
#     print(i/stem_data.student_attr_df['GENDER'].count())
# student_retention = pd.read_csv('/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/STUDENT_RETENTION_TABLE_full.csv')

In [ ]:
##### this has been revamped and moved to .py file
# def retention_helper(df):
#     #display(df)
#     last_major = df[['BA_DEGREE_RECEIVED','DEGREE_MAJOR1','DEGREE_MAJOR2','DEGREE_MAJOR3']].ffill().iloc[-1]
#     first_major = df[['DEGREE_MAJOR1','DEGREE_MAJOR2','DEGREE_MAJOR3']].bfill().iloc[0].rename({'DEGREE_MAJOR1' : 'INIT_MAJOR1',
#                                                                                                 'DEGREE_MAJOR2' : 'INIT_MAJOR2',
#                                                                                                 'DEGREE_MAJOR3' : 'INIT_MAJOR3'})
#     return pd.concat([first_major, last_major])
# #majors = stem_data.stu_retention_df.iloc[:100].groupby('PRSN_UNIV_ID').apply(retention_helper).head()
# #majors[majors['INIT_MAJOR1'] != majors['DEGREE_MAJOR1']]
# fake_student = student_retention[student_retention['PRSN_UNIV_ID']==127265]
# print(fake_student.columns)
# fake_student.iloc[0,8] = 'test'
# retention_helper(fake_student)

In [ ]:
## has been moved to .py
# stem_data.stu_retention_df['STEM_end'] = np.logical_or(stem_data.stu_retention_df['DEGREE_MAJOR1'].isin(stem_data.stem_major_df),
#              stem_data.stu_retention_df['DEGREE_MAJOR2'].isin(stem_data.stem_major_df))

# stem_data.stu_retention_df['STEM_end'] = np.logical_or(stem_data.stu_retention_df['STEM_end'],
#                                         stem_data.stu_retention_df['DEGREE_MAJOR3'].isin(stem_data.stem_major_df))

In [ ]:
#stem_data.stu_retention_df.reset_index()

In [ ]:
#print(stem_data.stem_major_df)
# list(stem_data.stu_retention_df['DEGREE_MAJOR1'].unique())
# info_major = np.where(stem_data.stu_retention_df['DEGREE_MAJOR1'] == 'Informatics BS')
# stem_data.stu_retention_df.iloc[info_major]['DEGREE_MAJOR1'].isin(stem_data.stem_major_df['STEM MAJORS'])

In [ ]:
#stem_data.stem_major_df

In [ ]:
## has been moved to .py
# stu_attribute_and_retention_df = pd.merge(stem_data.student_attr_df, stem_data.stu_retention_df.reset_index(), how = 'left',
#                                     left_on = 'PRSN_UNIV_ID', right_on = 'PRSN_UNIV_ID')

In [6]:
stem_data.stu_attribute_and_retention_df.columns

Index(['COHORT_SP_FA', 'COHORT_TERM_CD', 'COHORT_TERM_DESC',
       'ENTRY_ACADEMIC_LOAD', 'ENTRY_AGE', 'ENTRY_GPA', 'ENTRY_SCHOOL',
       'ENTRY_INTENDED_SCHOOL', 'ENTRY_PROGRAM', 'ENTRY_MAJOR_CODE',
       'ENTRY_MAJOR_DESC', 'ENTRY_UNITS', 'ETHNICITY', 'URM_FLAG',
       'FIRST_GENERATION', 'GENDER', 'HIGH_SCHOOL_GPA', 'NEW_STUDENT_CATEGORY',
       'PELL_ELIGIBILITY', 'PRSN_UNIV_ID', 'RESIDENCY', 'SAT_ACT_INDICATOR',
       'SAT_DRVD_COMPOSITE_SCORE_2019', 'TWNTYFRST_CENT_AWARD',
       'TWNTYFRST_CENT_SCHOLAR', 'CLASS_LEVEL', 'ENTRY_INTENDED_SCHOOL1',
       'ADMIT_STATUS', 'CERT_TERM_CD', 'TIME_2_CERT', 'STEM_start',
       'urm_flag_id', 'gender_id', 'pell_eligibility_id', 'INIT_MAJOR1',
       'INIT_MAJOR2', 'INIT_MAJOR3', 'BA_DEGREE_RECEIVED', 'DEGREE_MAJOR1',
       'DEGREE_MAJOR2', 'DEGREE_MAJOR3', 'STEM_end', 'MAINTAINED_MAJOR'],
      dtype='object')

In [ ]:
# i = stu_attribute_and_retention_df['ENTRY_MAJOR_DESC'] == stu_attribute_and_retention_df['DEGREE_MAJOR1']
# j = stu_attribute_and_retention_df['ENTRY_MAJOR_DESC'] == stu_attribute_and_retention_df['DEGREE_MAJOR2']
# k = stu_attribute_and_retention_df['ENTRY_MAJOR_DESC'] == stu_attribute_and_retention_df['DEGREE_MAJOR3']
# stu_attribute_and_retention_df['MAINTAINED_MAJOR'] = np.logical_or(i, np.logical_or(j, k))

In [7]:
stem_data.stu_attribute_and_retention_df.head()

,COHORT_SP_FA,COHORT_TERM_CD,COHORT_TERM_DESC,ENTRY_ACADEMIC_LOAD,ENTRY_AGE,ENTRY_GPA,ENTRY_SCHOOL,ENTRY_INTENDED_SCHOOL,ENTRY_PROGRAM,ENTRY_MAJOR_CODE,...,pell_eligibility_id,INIT_MAJOR1,INIT_MAJOR2,INIT_MAJOR3,BA_DEGREE_RECEIVED,DEGREE_MAJOR1,DEGREE_MAJOR2,DEGREE_MAJOR3,STEM_end,MAINTAINED_MAJOR
0,Fall,4068,Fall 2006,F,31,3.925,SCS,SCS,SCS1,GENSTBGS,...,0,History BA,NaN,NaN,1.0,History BA,NaN,NaN,False,False
1,Fall,4068,Fall 2006,F,19,2.700,COLL,COLL,COLL1,CMCLBA,...,1,Communication & Culture BA,NaN,NaN,1.0,Communication & Culture BA,NaN,NaN,False,True
2,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,BUSBSB,...,1,Finance BSB,Accounting BSB,NaN,1.0,Finance BSB,Accounting BSB,NaN,False,False
3,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,MARKBSB,...,1,Finance BSB,NaN,NaN,1.0,Finance BSB,NaN,NaN,False,False
4,Fall,4068,Fall 2006,F,18,0.000,BUS,BUS,BUS1,BEPPPBSB,...,1,Marketing BSB,Mathematics BA,NaN,1.0,Marketing BSB,Mathematics BA,NaN,True,False


In [ ]:
# has been moved
# def stem_generator_simple(self, dataset):
#     if dataset == 'train':
#         data = self.crs_df_train
#     elif dataset == 'valid':
#         data = self.crs_df_valid
#     else:
#         raise ValueError('Must be from valid or train data.')
        
#     for (student, term), df in self.crs_df_valid.groupby(['PRSN_UNIV_ID','ACAD_TERM_CD']):
#         courses_set = df['embedding_index']
#         if len(courses_set) > 1:
#             #print(student)
#             row = self.stu_attribute_and_retention_df[self.stu_attribute_and_retention_df['PRSN_UNIV_ID']==student]
#             student_status = row[['BA_DEGREE_RECEIVED', 'STEM_start', 'STEM_end']].values[0]
#             #print(student_status)
#             if student_status[1] == 0:
#                 continue
#             if student_status[0] == 0:
#                 y = 0 ## case 0 is dropout
#             else:
#                 if student_status[2] == 0:
#                     y = 1 ## case 1 transitioned out of STEM
#                 else:
#                     y = 2 ## completed STEM
#             x = self.embedding[courses_set].mean(axis=0)
#             yield x, y

In [52]:
stem_data.stem_major_df

,STEM MAJORS
0,Biology BS
1,Biology BA
2,Human Biology BS
3,Computer Science BS
4,Informatics BS
5,Neuroscience BS
6,Chemistry BS
7,Chemistry BA
8,Biochemistry BA
9,Environmantal Science BSES


In [ ]:
# print(stem_data.stu_attribute_and_retention_df['STEM_start'].sum())
# # stem_data.student_attr_df['ENTRY_MAJOR_DESC'].isin(stem_data.stem_major_df['STEM MAJORS']).sum()
# stem_data.student_attr_df['STEM_start'] = stem_data.student_attr_df['ENTRY_MAJOR_DESC'].isin(stem_data.stem_major_df['STEM MAJORS'])


In [ ]:
#stem_data._create_attribute_retention_table()

## stem simple RF

In [8]:
# for x,y in stem_generator_simple(stem_data):
#     print(x)

x_train_simp = []
y_train_simp = []
for x,y in stem_data.stem_generator_simple('train'):
    x_train_simp.append(x)
    y_train_simp.append(y)

In [9]:
#print(y_train)

In [10]:
x_train_simp = np.array(x_train_simp)
y_train_simp = np.array(y_train_simp)
print(x_train_simp.shape)
print(y_train_simp.shape)

clf = RandomForestClassifier()
clf.fit(x_train_simp, y_train_simp)

(12476, 78)
(12476,)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
y_pred_simp = clf.predict(x_train_simp)
print(y_pred_simp)
#confusion_matrix(y_train, y_pred)
clf.feature_importances_

[2 2 2 ... 0 0 0]


array([0.01537892, 0.01255356, 0.01122481, 0.01145042, 0.01128165,
       0.01188681, 0.01123348, 0.01150673, 0.01451663, 0.01551337,
       0.0156104 , 0.01167439, 0.01752281, 0.01453899, 0.01326366,
       0.01503575, 0.01084946, 0.01188622, 0.01114437, 0.01189565,
       0.01249756, 0.01170491, 0.01118347, 0.01411914, 0.01219421,
       0.01374615, 0.01414745, 0.01252448, 0.01183544, 0.01219513,
       0.01099836, 0.01262547, 0.01138632, 0.0140481 , 0.01434252,
       0.01230517, 0.01038116, 0.02206495, 0.01128735, 0.01296411,
       0.01145871, 0.01332157, 0.01061378, 0.01411649, 0.01194147,
       0.0132215 , 0.01185361, 0.01267209, 0.01485921, 0.01212138,
       0.01082741, 0.01407528, 0.00965438, 0.01204101, 0.01076677,
       0.01196328, 0.01555786, 0.01671075, 0.01469429, 0.01114764,
       0.01197071, 0.01164291, 0.01142667, 0.01526375, 0.01740823,
       0.01479521, 0.01286503, 0.01037272, 0.01147486, 0.01100491,
       0.01244025, 0.01327265, 0.01066109, 0.01136367, 0.01342

In [12]:
confusion_matrix(y_train_simp, y_pred_simp)

array([[4368,    5,   17],
       [  11, 2849,   25],
       [  18,    7, 5176]])

In [13]:
clf.score(x_train_simp, y_train_simp)

0.9933472266752165

In [14]:
# for x,y in stem_generator_simple(stem_data):
#     print(x)

x_valid_simp = []
y_valid_simp = []
for x,y in stem_data.stem_generator_simple('valid'):
    x_valid_simp.append(x)
    y_valid_simp.append(y)

In [15]:
y_pred_simp = clf.predict(x_valid_simp)
print(confusion_matrix(y_valid_simp, y_pred_simp))
print(clf.feature_importances_)
clf.score(x_valid_simp, y_valid_simp)

[[465  79 358]
 [235 173 211]
 [257  49 659]]
[0.01537892 0.01255356 0.01122481 0.01145042 0.01128165 0.01188681
 0.01123348 0.01150673 0.01451663 0.01551337 0.0156104  0.01167439
 0.01752281 0.01453899 0.01326366 0.01503575 0.01084946 0.01188622
 0.01114437 0.01189565 0.01249756 0.01170491 0.01118347 0.01411914
 0.01219421 0.01374615 0.01414745 0.01252448 0.01183544 0.01219513
 0.01099836 0.01262547 0.01138632 0.0140481  0.01434252 0.01230517
 0.01038116 0.02206495 0.01128735 0.01296411 0.01145871 0.01332157
 0.01061378 0.01411649 0.01194147 0.0132215  0.01185361 0.01267209
 0.01485921 0.01212138 0.01082741 0.01407528 0.00965438 0.01204101
 0.01076677 0.01196328 0.01555786 0.01671075 0.01469429 0.01114764
 0.01197071 0.01164291 0.01142667 0.01526375 0.01740823 0.01479521
 0.01286503 0.01037272 0.01147486 0.01100491 0.01244025 0.01327265
 0.01066109 0.01136367 0.01342665 0.01165993 0.01481436 0.01199839]


0.5217216411906678

In [88]:
stem_data.stu_retention_df[stem_data.stu_retention_df['BA_DEGREE_RECEIVED']==0]

,INIT_MAJOR1,INIT_MAJOR2,INIT_MAJOR3,BA_DEGREE_RECEIVED,DEGREE_MAJOR1,DEGREE_MAJOR2,DEGREE_MAJOR3,STEM_end
PRSN_UNIV_ID,,,,,,,,
80319,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False
101701,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False
119007,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False
152200,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False
161801,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...
2000758069,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False
2000758315,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False
2000758317,NaN,NaN,NaN,0.0,NaN,NaN,NaN,False


In [89]:
stu_retention_raw = pd.read_csv('/Users/rsciagli/iCloud Drive (Archive)/Documents/Documents - Rebecca’s MacBook Air/fall2020/bar/STUDENT_RETENTION_TABLE_full.csv')

In [90]:
stu_retention_raw_df = pd.DataFrame(stu_retention_raw)

In [148]:
temp = stu_retention_raw_df[['BA_DEGREE_RECEIVED','DEGREE_MAJOR1']]
temp = pd.DataFrame(temp)

In [150]:
temp[temp['DEGREE_MAJOR1']=='Biology BS'].where()
x = temp['DEGREE_MAJOR1']=='Biology BS'
find = np.where(temp[temp['DEGREE_MAJOR1']=='Biology BS'])

,BA_DEGREE_RECEIVED,DEGREE_MAJOR1
1063,1,Biology BS
1064,1,Biology BS
1065,1,Biology BS
1066,1,Biology BS
1067,1,Biology BS
...,...,...
1620364,1,Biology BS
1660499,1,Biology BS
1660500,1,Biology BS
1660501,1,Biology BS


In [145]:
for i in stu_retention_raw_df:
    if temp['BA_DEGREE_RECEIVED'] == 0:
        print(i)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
for student stu_retention_raw_df:
    #row = stu_retention_raw_df[stu_retention_raw_df['PRSN_UNIV_ID']==student]
    temp = stu_retention_raw_df[stu_retention_raw_df['DEGREE_MAJOR1']=='Biology BS']
    print(temp)
        

In [ ]:
#stem_data.stu_retention_df[stem_data.stu_retention_df['INIT_MAJOR1'] == 'Computer Science BS']

In [42]:
def stem_generator_complex(self, dataset, major_filter=None):
    if dataset == 'train':
        data = self.crs_df_train
    elif dataset == 'valid':
        data = self.crs_df_valid
    else:
        raise ValueError('Must be from valid or train data.')
        
    for (student, term), df in data.groupby(['PRSN_UNIV_ID','ACAD_TERM_CD']):
        df = df[~df['CRS_OFCL_GRD_NBR'].isna()]
        df = df[df['ACAD_UNT_TKN_NBR'] > 0]
        courses_set = df['embedding_index']
        #print(df)
        if len(courses_set) > 1:
            row = self.stu_attribute_and_retention_df[self.stu_attribute_and_retention_df['PRSN_UNIV_ID']==student] 
            major = row['INIT_MAJOR1'].values[0]
            if  major_filter is not None and major_filter != major:
                continue
            #x = self.embedding[courses_set].mean(axis=0)
            df['grade_points'] = df['CRS_OFCL_GRD_NBR']*df['ACAD_UNT_TKN_NBR']
            #print(df['grade_points'])
            ave_grade = df['grade_points'].sum()/df['ACAD_UNT_TKN_NBR'].sum()
            var_grade = ((df['ACAD_UNT_TKN_NBR']/df['ACAD_UNT_TKN_NBR'].sum()) * 
                         np.square(df['CRS_OFCL_GRD_NBR']-ave_grade)).sum()
            credit_hrs = df['ACAD_UNT_TKN_NBR'].sum()
            #print(credit_hrs)
            grade_scalars = np.array([ave_grade, var_grade, credit_hrs]).astype(float)
            #print(grade_scalars)
            
            df_low = df[df['CRS_OFCL_GRD_NBR'] <= ave_grade + 0.001]
            #print(df_low.shape)
            df_high = df[df['CRS_OFCL_GRD_NBR'] >= ave_grade - 0.001]
            if len(df_high) == 0:
                display(df)
                print(df['CRS_OFCL_GRD_NBR'])
                print(grade_scalars)
            assert len(df_high) > 0
            assert len(df_low) > 0
            x_low = self.embedding[df_low['embedding_index']].mean(axis=0)
            x_high = self.embedding[df_high['embedding_index']].mean(axis=0)
                   
            #ethnicity_onehot = np.eye(len(self.ethnicity_id))[row['ethnicity_id']][0]
            urm_onehot = np.eye(len(self.urm_flag_id))[row['urm_flag_id']][0]
            gender_onehot = np.eye(len(self.gender_id))[row['gender_id']][0]
            pell_eligibility = row['pell_eligibility_id']

            x = np.concatenate([x_low, x_high, grade_scalars, urm_onehot, gender_onehot, pell_eligibility])
            #x = np.concatenate([x, ethnicity_onehot, urm_onehot, gender_onehot, pell_eligibility])
            student_status = row[['BA_DEGREE_RECEIVED', 'STEM_start', 'STEM_end']].values[0]
            if student_status[1] == 0:
                continue
            if student_status[0] == 0:
                y = 0 ## case 0 is degree non-completer
            else:
                if student_status[2] == 0:
                    y = 1 ## case 1 STEM non-completer
                else:
                    y = 2 ## case 2 STEM completer
                
            yield x, y
            
            if urm_onehot[0] == 0 and dataset == 'train': ## oversampling non-whites
                yield x, y

In [43]:
#stem_data.stem_major_df

## stem complex RF

In [60]:
# x_train_complex = []
# y_train_complex = []
# for x,y in stem_generator_complex(stem_data, 'train'):
#     x_train_complex.append(x)
#     y_train_complex.append(y)

In [67]:
#np.save('x_train_complex.npy', x_train_complex)

In [ ]:
np.save('x_train_complex.npy', x_train_complex)

In [68]:
#np.save('y_train_complex.npy', y_train_complex)

In [ ]:
np.save('y_train_complex.npy', y_train_complex)

In [81]:
x_train_complex = np.array(x_train_complex)
y_train_complex = np.array(y_train_complex)
print(x_train_complex.shape)
print(x_train_complex.dtype)
print(y_train_complex.shape)

clf = RandomForestClassifier(max_depth=7)
clf.fit(x_train_complex, y_train_complex)

(12314, 166)
float64
(12314,)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [82]:
y_pred_complex = clf.predict(x_train_complex)
print(confusion_matrix(y_train_complex, y_pred_complex))
print(clf.feature_importances_)
clf.score(x_train_complex, y_train_complex)

[[2642   39 1609]
 [1066  558 1234]
 [1254   21 3891]]
[0.00563679 0.0021437  0.00332128 0.00243337 0.0054473  0.00840963
 0.00267011 0.0041245  0.01108343 0.01820466 0.0078198  0.00493733
 0.01390902 0.00516285 0.00735419 0.00762272 0.00274522 0.00179628
 0.00553247 0.00374264 0.00415008 0.00387098 0.00187145 0.01486469
 0.00665081 0.01268141 0.00525934 0.00318908 0.00663197 0.00612564
 0.00393631 0.00574721 0.00411352 0.01690424 0.00628965 0.00473104
 0.00356575 0.02606471 0.00418638 0.00502797 0.00300811 0.00237915
 0.00410312 0.00859576 0.00724153 0.00541538 0.00780872 0.00881269
 0.00756517 0.00230449 0.00197465 0.00943779 0.00226519 0.00248119
 0.00216655 0.00219015 0.01083631 0.00907668 0.00484006 0.00269004
 0.00243875 0.00257848 0.00259849 0.02159921 0.02175842 0.02204182
 0.01430605 0.00246815 0.00306563 0.00258195 0.00220262 0.00769968
 0.00156202 0.00309451 0.0106996  0.00306624 0.00736164 0.00890806
 0.00720572 0.00284459 0.00279637 0.00247245 0.00271232 0.00296284
 0.0018

0.5758486275783661

In [76]:
# x_valid_complex = []
# y_valid_complex = []
# for x,y in stem_generator_complex(stem_data, 'valid'):
#     x_valid_complex.append(x)
#     y_valid_complex.append(y)

In [69]:
np.save('x_valid_complex.npy', x_valid_complex)
np.save('y_valid_complex.npy', y_valid_complex)

In [83]:
y_pred_complex = clf.predict(x_valid_complex)
print(confusion_matrix(y_valid_complex, y_pred_complex))
clf.feature_importances_

[[488  22 374]
 [267  71 275]
 [267  20 670]]


array([0.00563679, 0.0021437 , 0.00332128, 0.00243337, 0.0054473 ,
       0.00840963, 0.00267011, 0.0041245 , 0.01108343, 0.01820466,
       0.0078198 , 0.00493733, 0.01390902, 0.00516285, 0.00735419,
       0.00762272, 0.00274522, 0.00179628, 0.00553247, 0.00374264,
       0.00415008, 0.00387098, 0.00187145, 0.01486469, 0.00665081,
       0.01268141, 0.00525934, 0.00318908, 0.00663197, 0.00612564,
       0.00393631, 0.00574721, 0.00411352, 0.01690424, 0.00628965,
       0.00473104, 0.00356575, 0.02606471, 0.00418638, 0.00502797,
       0.00300811, 0.00237915, 0.00410312, 0.00859576, 0.00724153,
       0.00541538, 0.00780872, 0.00881269, 0.00756517, 0.00230449,
       0.00197465, 0.00943779, 0.00226519, 0.00248119, 0.00216655,
       0.00219015, 0.01083631, 0.00907668, 0.00484006, 0.00269004,
       0.00243875, 0.00257848, 0.00259849, 0.02159921, 0.02175842,
       0.02204182, 0.01430605, 0.00246815, 0.00306563, 0.00258195,
       0.00220262, 0.00769968, 0.00156202, 0.00309451, 0.01069

In [84]:
print(classification_report(y_valid_complex, y_pred_complex))

              precision    recall  f1-score   support

           0       0.48      0.55      0.51       884
           1       0.63      0.12      0.20       613
           2       0.51      0.70      0.59       957

    accuracy                           0.50      2454
   macro avg       0.54      0.46      0.43      2454
weighted avg       0.53      0.50      0.46      2454



In [85]:
clf.score(x_valid_complex, y_valid_complex)

0.5008149959250203

In [ ]:
## pulling out specific course embeddings
core_courses = [108, 136, 196]   ## these are embedding ids
baseline = stem_data.embedding[core_courses].sum(axis=0) #ID vector
x_temp = stem_data.embedding + baseline.reshape(1,-1) / (1+len(core_courses))

In [ ]:
pred = clf.predict(x_temp)
dropout_courses = np.arange(len(stem_data.embedding))[pred==0]  # 0 is dropout?

# pred_proba = clf.predict_proba(x_temp)

In [ ]:
crs_id_to_course_name = dict(stem_data.crs_df[['agg_id', 'CRS_NM']].values)

In [27]:
stem_data.course_to_id['11262.0']

1777

In [29]:
## this gives the embedding id for the course
print(stem_data.course_to_id[stem_data.name_to_course['MATH-M211']])
## name_to_course returns the course ID (agg_id) of for the given course
print(stem_data.name_to_course['MATH-M211'])

136
31133.0


In [ ]:
crs_id_to_course_name['11262.0']

## socio-demo only RF

In [ ]:
x_train_demo = []
y_train_demo = []
for x,y in stem_data.socio_demo_generator_complex('train'):
    x_train_demo.append(x)
    y_train_demo.append(y)

In [ ]:
x_train_demo = np.array(x_train_demo)
y_train_demo = np.array(y_train_demo)
print(x_train_demo.shape)
print(x_train_demo.dtype)
print(y_train_demo.shape)

clf = RandomForestClassifier()
clf.fit(x_train_demo, y_train_demo)

In [ ]:
y_pred_demo = clf.predict(x_train_demo)
print(confusion_matrix(y_train_demo, y_pred_demo))
clf.feature_importances_
clf.score(x_train_demo, y_train_demo)

In [ ]:
x_valid_demo = []
y_valid_demo = []
for x,y in stem_data.socio_demo_generator_complex('valid'):
    x_valid_demo.append(x)
    y_valid_demo.append(y)

In [ ]:
y_pred_demo = clf.predict(x_valid_demo)
print(confusion_matrix(y_valid_demo, y_pred_demo))
clf.feature_importances_

In [ ]:
clf.score(x_valid_demo, y_valid_demo)

In [ ]:
print(stem_data.student_attr_df['URM_FLAG'].count())
print(stem_data.student_attr_df['URM_FLAG'].value_counts())
print(stem_data.student_attr_df['URM_FLAG'].unique())

In [ ]:
# def socio_demo_generator_complex(self):
#     for (student, term), df in self.crs_df_valid.groupby(['PRSN_UNIV_ID','ACAD_TERM_CD']):
#         df = df[~df['CRS_OFCL_GRD_NBR'].isna()]
#         courses_set = df['embedding_index'] 
#         if len(courses_set) > 1:        
#             row = self.stu_attribute_and_retention_df[self.stu_attribute_and_retention_df['PRSN_UNIV_ID']==student]        
#             ethnicity_onehot = np.eye(len(self.ethnicity_id))[row['ethnicity_id']][0]
#             urm_onehot = np.eye(len(self.urm_flag_id))[row['urm_flag_id']][0]
#             gender_onehot = np.eye(len(self.gender_id))[row['gender_id']][0]
#             pell_eligibility = row['pell_eligibility_id']
            
#             x = np.concatenate([ethnicity_onehot, urm_onehot, gender_onehot, pell_eligibility])          
#             student_status = row[['BA_DEGREE_RECEIVED', 'STEM_start', 'STEM_end']].values[0]
#             if student_status[1] == 0:
#                 continue
#             if student_status[0] == 0:
#                 y = 0 ## case 0 is degree non-completer
#             else:
#                 if student_status[2] == 0:
#                     y = 1 ## case 1 STEM non-completer
#                 else:
#                     y = 2 ## case 2 STEM completer
#             yield x, y

In [ ]:
print(len(stem_data.ethnicity_id))
print(len(stem_data.urm_flag_id))
print(len(stem_data.gender_id))
print(len(stem_data.pell_eligibility_id))


In [ ]:
# self.gpao_id = list(self.crs_df['GPAO'].unique())
# self.gpao_to_id = dict([(gpao, i) for i, gpao in enumerate(self.gpao_id)])
# self.course_grade_id = list(self.crs_df['CRS_OFCL_GRD_NBR'].unique())
# self.course_grade_to_id = dict([(grade, i) for i, grade in enumerate(self.course_grade_id)])
# self.credit_hrs_id = list(self.crs_df['ACAD_UNT_TKN_NBR'].unique())
# self.credit_hrs_to_id = dict([(hours, i) for i, hours in enumerate(self.credit_hrs_id)])

# self.crs_df['gpao_id'] = self.crs_df['GPAO'].map(self.gpao_to_id)
# self.crs_df['course_grade_id'] = self.crs_df['CRS_OFCL_GRD_NBR'].map(self.course_grade_to_id)
# self.crs_df['credit_hrs_id'] = self.crs_df['ACAD_UNT_TKN_NBR'].map(self.credit_hrs_to_id)

self.sat_id = list(self.student_attr_df['SAT_DRVD_COMPOSITE_SCORE_2019'].unique())
self.sat_to_id = dict([(sat, i) for i, sat in enumerate(self.sat_id)])


self.student_attr_df['sat_id'] = self.student_attr_df['SAT_DRVD_COMPOSITE_SCORE_2019'].map(self.sat_to_id)

In [ ]:
print(stem_data.crs_df['GPAO'].isnull().value_counts())
print(stem_data.crs_df['GPAO'].nunique())
print(stem_data.crs_df['GPAO'].count())
stem_data.crs_df['GPAO'].value_counts()

In [ ]:
print(stem_data.crs_df['CRS_OFCL_GRD_NBR'].isnull().value_counts())
print(stem_data.crs_df['CRS_OFCL_GRD_NBR'].nunique())
print(stem_data.crs_df['CRS_OFCL_GRD_NBR'].count())
stem_data.crs_df['CRS_OFCL_GRD_NBR'].value_counts()

In [ ]:
print(stem_data.crs_df['ACAD_UNT_TKN_NBR'].isnull().value_counts())
print(stem_data.crs_df['ACAD_UNT_TKN_NBR'].nunique())
print(stem_data.crs_df['ACAD_UNT_TKN_NBR'].count())
stem_data.crs_df['ACAD_UNT_TKN_NBR'].value_counts()

In [ ]:
stem_data.student_attr_df.columns

In [ ]:
print(stem_data.student_attr_df['SAT_DRVD_COMPOSITE_SCORE_2019'].isnull().value_counts())
print(stem_data.student_attr_df['SAT_DRVD_COMPOSITE_SCORE_2019'].nunique())
print(stem_data.student_attr_df['SAT_DRVD_COMPOSITE_SCORE_2019'].count())
print(stem_data.student_attr_df['SAT_DRVD_COMPOSITE_SCORE_2019'].value_counts())

# print(stem_data.student_attr_df['SAT_ACT_INDICATOR'].isnull().value_counts())
# print(stem_data.student_attr_df['SAT_ACT_INDICATOR'].nunique())
# print(stem_data.student_attr_df['SAT_ACT_INDICATOR'].count())
# stem_data.student_attr_df['SAT_ACT_INDICATOR'].value_counts()

In [ ]:
#stem_data.student_attr_df['STEM_start'] = stem_data.student_attr_df['ENTRY_MAJOR_DESC'].isin(stem_major_df)

In [ ]:
#stem_data.student_attr_df.head()

In [ ]:
#list(student_retention.columns)
#stem_data.student_attr_df[stem_data.student_attr_df['ENTRY_MAJOR_DESC']=='Economics BA']

In [ ]:
def major_and_school_helper(df):
    return df[['ENTRY_INTENDED_SCHOOL','ENTRY_MAJOR_DESC']].iloc[-1]
#stem_data.student_attr_df.iloc[51:100].groupby('PRSN_UNIV_ID').apply(major_and_school_helper)

In [ ]:
print(stu_retention_df['DEGREE_MAJOR1'].isnull().sum())
stu_retention_df['BA_DEGREE_RECEIVED'].isnull().sum()

In [ ]:
stu_retention_df['DEGREE_MAJOR2'].isnull().value_counts()

In [ ]:
stu_retention_df['DEGREE_MAJOR3'].isnull().value_counts()

In [ ]:
stem_data.student_attr_df['ENTRY_INTENDED_SCHOOL'].isnull().sum()

In [ ]:
print(stem_data.crs_df[stem_data.crs_df['PRSN_UNIV_ID']==1996190])
stem_data.crs_df['PRSN_UNIV_ID'][7000]

In [ ]:
#stem_data.student_attr_df.iloc[0]
print(stem_data.student_attr_df['ENTRY_MAJOR_CODE'].isnull().sum())
print(stem_data.student_attr_df['ENTRY_INTENDED_SCHOOL'].isnull().sum())
stem_data.student_attr_df['ENTRY_MAJOR_DESC'].isnull().sum()

In [ ]:
## this takes in a course id and finds its associated course name
def course_to_label(course):
    return stem_data.crs_df[stem_data.crs_df['agg_id']==course].iloc[0]['CRS_NM']

In [ ]:
course_to_label(stem_data.embedding_id[210])

In [ ]:
dept_with_orphans = stem_data.crs_df[stem_data.crs_df['agg_id']==stem_data.crs_df['CRS_SUBJ_DEPT_CD']]['agg_id'].unique()
stem_data.course_to_id['other_dept']

In [ ]:
## argsort gives us courses that are similar in the order of least like to most like of all the embedding ids
np.argsort(model_embeddings[3]@model_embeddings.T)
print(course_to_label(stem_data.embedding_id[4]))
print(course_to_label(stem_data.embedding_id[123]))

In [ ]:
## this gives the cosine similarity (dot) for two courses.  the larger the number the higher the correlation,
## negative value means anticorrelated. zero means no correlation.
## we will need to normalize the vectors at some point.
model_embeddings[3]@model_embeddings[123]

In [31]:
def course_cosine_similarity(course1, course2):
    crs1 = course_to_label(stem_data.embedding_id[course1])
    crs2 = course_to_label(stem_data.embedding_id[course2])
    vec_1 = model_embeddings[course1]
    vec_2 = model_embeddings[course2]
    correlation = np.dot(vec_1, vec_2)/np.linalg.norm(vec_1)/np.linalg.norm(vec_2)
    
    return crs1, crs2, correlation     

In [32]:
course_cosine_similarity(136,3026)

NameError: name 'course_to_label' is not defined

In [ ]:
def course_correlation(crs):
    correlation = np.argsort(model_embeddings[crs]@model_embeddings.T)
    #name = course_to_label(stem_data.embedding_id[course1])
    vec_1 = model_embeddings[crs]
    correlation = np.dot(model_embeddings, vec_1)/np.linalg.norm(vec_1)/np.linalg.norm(model_embeddings, axis=1)
    #return list(sorted(correlation))[-20:]
    highest_matches = np.argsort(correlation)[-10:]
    for i in highest_matches:
        print(i, course_to_label(stem_data.embedding_id[i]), correlation[i])
        #return(vec_1, i, course_to_label(stem_data.embedding_id[i]), correlation[i])
        
    lowest_matches = np.argsort(correlation)[:10]
    for i in lowest_matches:
        print(i, course_to_label(stem_data.embedding_id[i]), correlation[i])
        #return(vec_1, i, course_to_label(stem_data.embedding_id[i]), correlation[i])

In [ ]:
#normalized_embeddings = model_embeddings/np.linalg.norm(model_embeddings, axis=1)

In [ ]:
name_to_course = dict(stem_data.crs_df[['CRS_NM','agg_id']].values)

In [30]:
course_cosine_similarity(3225,5511)

NameError: name 'course_cosine_similarity' is not defined

In [ ]:
dif_a = model_embeddings[1122]-model_embeddings[2188]
dif_b = model_embeddings[1617]-model_embeddings[85]
#dif_c = model_embeddings[84]-model_embeddings[869]

In [ ]:
np.dot(dif_a, dif_b)/np.linalg.norm(dif_a)/np.linalg.norm(dif_b)

In [ ]:
df_model_embeddings = pd.DataFrame(model_embeddings[[3026,5303,290,3225,5881,280,2188,1195,333,3425]].T,
                                                    columns=['CSCI-C241','CSCI-H241','CSCI-A290','CSCI-B351',
                                                    'CSCI-B365','BUS-F260','FRIT-M200','MATH-M301','MATH-M303',
                                                    'MATH-S311'])                                                                                                                                               

In [ ]:
#df_model_embeddings.corr()
#df_model_embeddings.corr().to_csv('CHEM_BS_FIRST4SEM_UGcurr',index=True, sep='\t')

In [ ]:
#kot = corr[corr>=.2]
# sb.set(context='notebook', style='white', palette='deep', font='Avenir', font_scale=1.2, color_codes=True, rc=None)
# plt.figure(figsize=(12,10))
# plt.title('Third Semester Math and Computer Science course cosine similarities')
# sb.heatmap(df_model_embeddings.corr(), annot=True, cmap='Greens')

In [ ]:
student_attr = pd.read_excel('/Users/rsciagli/documents/Fall2020/BAR/student_attribute_table_full.xlsx')

In [ ]:
stem_data.student_attr_df.columns

In [ ]:
student_attr_df = pd.DataFrame(student_attr)

In [ ]:
student_attr_df['ENTRY_MAJOR_DESC']

In [ ]:
#stem_data.student_attr_df[stem_data.student_attr_df['ETHNICITY']=='NOANSWER']

In [ ]:
# relocated_null_ethnicity = stem_data.student_attr_df[stem_data.student_attr_df['agg_id']==stem_data.student_attr_df['CRS_SUBJ_DEPT_CD']]['agg_id'].unique()

#null_ethnicity = stem_data.student_attr_df['ETHNICITY'].isnull()
stem_data.student_attr_df['ETHNICITY'] = np.where(stem_data.student_attr_df['ETHNICITY'].isnull(), 
                                                  'NOANSWER', stem_data.student_attr_df['ETHNICITY'])
    

In [ ]:
## I want to collapse 
print(stem_data.student_attr_df['ETHNICITY'].count())
print(stem_data.student_attr_df['ETHNICITY'].value_counts())
print(stem_data.student_attr_df['ETHNICITY'].unique())


In [ ]:
ethnicity_id = list(student_attr_df['ETHNICITY'].unique())
ethnicity_to_id = dict([(ethnicity, i) for i, ethnicity in enumerate(ethnicity_id)])

In [ ]:
stem_data.student_attr_df['PELL_ELIGIBILITY'].isnull().sum()

In [ ]:
urm_flag_id = list(student_attr_df['URM_FLAG'].unique())
urm_flag_to_id = dict([(urm, i) for i, urm in enumerate(urm_flag_id)])

In [ ]:
gender_id = list(student_attr_df['GENDER'].unique())
gender_to_id = dict([(gender, i) for i, gender in enumerate(gender_id)])

In [ ]:
stem_data.student_attr_df['GENDER'].value_counts()

In [ ]:
pell_eligibility_id = list(student_attr_df['PELL_ELIGIBILITY'].unique())
pell_status_to_id = dict([(status, i) for i, status in enumerate(pell_eligibility_id)])

In [ ]:
stem_data.student_attr_df['PELL_ELIGIBILITY'].nunique()

In [ ]:
major_id = list(student_attr_df['ENTRY_MAJOR_DESC'].unique())
major_to_id = dict([(major, i) for i, major in enumerate(major_id)])

intended_school_id = list(student_attr_df['ENTRY_INTENDED_SCHOOL'].unique())
indended_school_to_id = dict([(pre_major, i) for i, pre_major in enumerate(intended_school_id)])

In [ ]:
student_major = pd.read_csv('/Users/rsciagli/documents/Fall2020/BAR/STU_MAJOR_TABLE_full.csv')
student_major.head()